# Merge Metadata

This script can be used to build the final Metadata file.

There are several notes that are important:

* Some of the location files have been processed externally and the workflow needs to be explained here
* We should note done all CRS transformations applied for reference 

In [2]:
import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

from camelsp import Bundesland, util

As an example: The `Bundesland` context manager can load the metadata for the given Bundesland only from the full metadata table. If this table does not yet exist, it is created from the NUTSID mapping table. Check out for Saarland:

In [3]:
with Bundesland('DEC') as bl:
    dec_meta = bl.metadata

dec_meta.head()

,camels_id,provider_id,camels_path,nuts_lvl2,federal_state,area,x,y,q_count,w_count,q_w_pearson,q_w_spearman
1372,DEC10000,1271120,./DEC/DEC10000/DEC10000_data.csv,DEC,Saarland,"44,200km²",4.095083e+06,2.947311e+06,22707.0,22707.0,0.925040,0.982478
1373,DEC10010,1122120,./DEC/DEC10010/DEC10010_data.csv,DEC,Saarland,203km²,4.090150e+06,2.907085e+06,21611.0,21611.0,0.927440,0.897286
1374,DEC10020,1482120,./DEC/DEC10020/DEC10020_data.csv,DEC,Saarland,4671km²,4.085716e+06,2.914696e+06,0.0,0.0,NaN,NaN
1375,DEC10030,1251120,./DEC/DEC10030/DEC10030_data.csv,DEC,Saarland,"12,500km²",4.110659e+06,2.943239e+06,6909.0,6909.0,0.924174,0.997792
1376,DEC10040,1071120,./DEC/DEC10040/DEC10040_data.csv,DEC,Saarland,"48,40km²",4.102207e+06,2.948140e+06,0.0,0.0,NaN,NaN


## Generate basic metadata

This step will produce one metadata file containing all processed data, which can be used as NUTS lookup and as a basis to add more specific metadata.
The first step also loads the Location files and merges everything

In [4]:
for NUTS in tqdm(util._NUTS_LVL2_NAMES.keys()):
    with Bundesland(NUTS) as bl:
        try:
            p = os.path.join(bl.base_path, 'locations', f'{bl.NUTS}_Locations.csv')
            # read in 
            df = pd.read_csv(p)
            df.columns = ['provider_id', 'area', 'x', 'y']
        except FileNotFoundError:
            continue
        
        # update by simply setting the new metadata to the property setter
        # in this case, the joining column needs to be 'camels_id' or 'provider_id'
        bl.metadata = df

        # or use the function if you prefer
        #bl.update_metadata(df, id_column='provider_id')

metadata = util.get_metadata()
metadata

100%|██████████| 16/16 [00:00<00:00, 23.06it/s]


,provider_id,camels_id,camels_path,nuts_lvl2,federal_state,area,x,y,q_count,w_count,q_w_pearson,q_w_spearman
0,57300.0,DEG10000,./DEG/DEG10000/DEG10000_data.csv,DEG,Thüringen,NaN,NaN,NaN,29646.0,29646.0,0.969240,0.976895
1,44700.0,DEG10010,./DEG/DEG10010/DEG10010_data.csv,DEG,Thüringen,NaN,NaN,NaN,22707.0,22707.0,NaN,NaN
2,57420.0,DEG10020,./DEG/DEG10020/DEG10020_data.csv,DEG,Thüringen,NaN,NaN,NaN,35490.0,35490.0,0.958767,0.962387
3,57650.0,DEG10030,./DEG/DEG10030/DEG10030_data.csv,DEG,Thüringen,NaN,NaN,NaN,12845.0,12845.0,0.502141,0.553354
4,57021.0,DEG10040,./DEG/DEG10040/DEG10040_data.csv,DEG,Thüringen,NaN,NaN,NaN,21246.0,21246.0,0.940139,0.977699
...,...,...,...,...,...,...,...,...,...,...,...,...
3655,56113404,DE215350,./DE2/DE215350/DE215350_data.csv,DE2,Bayern,8.2,4.456659e+06,3009715.476,16497.0,16497.0,0.558268,0.933178
3656,56114000,DE215360,./DE2/DE215360/DE215360_data.csv,DE2,Bayern,14.1,4.457008e+06,3011664.641,20880.0,20880.0,0.996458,0.985672
3657,56122008,DE215370,./DE2/DE215370/DE215370_data.csv,DE2,Bayern,84.3,4.465052e+06,3016792.655,23072.0,23072.0,0.399310,0.898170
3658,56143008,DE215380,./DE2/DE215380/DE215380_data.csv,DE2,Bayern,92.4,4.462157e+06,3021202.262,23440.0,23440.0,0.547284,0.871654


## Count existing data

Go for each file and count the available data. Add anything that makes it necessary to read every single data file into the for-loop below.

Checking columns:

* `'q'` 
* `'w'`

In [5]:
for NUTS in util._NUTS_LVL2_NAMES.keys():
    # empty container for this BL
    count_q = []
    count_w = []
    
    # process this federal state
    with Bundesland(NUTS) as bl:
        # get meta
        meta = bl.metadata

        # go for each id
        for camels_id in tqdm(meta.camels_id.values):
            # load the data
            try:
                df = bl.get_data(camels_id)
            except FileNotFoundError:
                count_q.append(0)
                count_w.append(0)
                continue
    
            # check q
            if 'q' in df.columns.values:
                count_q.append((~df.q.isna()).count())
            else:
                count_q.append(0)

            # check w
            if 'w' in df.columns.values:
                count_w.append((~df.w.isna()).count())
            else:
                count_w.append(0)

        # build the new metadata
        counts = pd.DataFrame({'camels_id': meta.camels_id.values, 'q_count': np.asarray(count_q, dtype=int), 'w_count': np.asarray(count_w, dtype=int)})

        # add to metadata
        bl.update_metadata(counts)

metadata = util.get_metadata()
metadata

100%|██████████| 540/540 [00:21<00:00, 24.81it/s]
0it [00:00, ?it/s]
 16%|█▌        | 60/382 [00:02<00:10, 29.37it/s]/home/mirko/Dropbox/python/camelsp/camelsp/output.py:337: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, parse_dates=['date'])
100%|██████████| 382/382 [00:08<00:00, 42.58it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 437/437 [00:09<00:00, 47.70it/s] 
0it [00:00, ?it/s]
100%|██████████| 63/63 [00:03<00:00, 19.95it/s]


,camels_id,provider_id,camels_path,nuts_lvl2,federal_state,area,x,y,q_count,w_count,q_w_pearson,q_w_spearman
0,DEG10000,57300.0,./DEG/DEG10000/DEG10000_data.csv,DEG,Thüringen,NaN,NaN,NaN,29646.0,29646.0,0.969240,0.976895
1,DEG10010,44700.0,./DEG/DEG10010/DEG10010_data.csv,DEG,Thüringen,NaN,NaN,NaN,22707.0,22707.0,NaN,NaN
2,DEG10020,57420.0,./DEG/DEG10020/DEG10020_data.csv,DEG,Thüringen,NaN,NaN,NaN,35490.0,35490.0,0.958767,0.962387
3,DEG10030,57650.0,./DEG/DEG10030/DEG10030_data.csv,DEG,Thüringen,NaN,NaN,NaN,12845.0,12845.0,0.502141,0.553354
4,DEG10040,57021.0,./DEG/DEG10040/DEG10040_data.csv,DEG,Thüringen,NaN,NaN,NaN,21246.0,21246.0,0.940139,0.977699
...,...,...,...,...,...,...,...,...,...,...,...,...
3655,DE215350,56113404,./DE2/DE215350/DE215350_data.csv,DE2,Bayern,8.2,4.456659e+06,3009715.476,16497.0,16497.0,0.558268,0.933178
3656,DE215360,56114000,./DE2/DE215360/DE215360_data.csv,DE2,Bayern,14.1,4.457008e+06,3011664.641,20880.0,20880.0,0.996458,0.985672
3657,DE215370,56122008,./DE2/DE215370/DE215370_data.csv,DE2,Bayern,84.3,4.465052e+06,3016792.655,23072.0,23072.0,0.399310,0.898170
3658,DE215380,56143008,./DE2/DE215380/DE215380_data.csv,DE2,Bayern,92.4,4.462157e+06,3021202.262,23440.0,23440.0,0.547284,0.871654


## Add W ~ Q correlations 

The data reports contain correlations between all data variables. We can extract the pearson's correlation coefficient or the spearman rank correlation and add to the preliminary metadata file

In [6]:
util.get_metadata()

,camels_id,provider_id,camels_path,nuts_lvl2,federal_state,area,x,y,q_count,w_count,q_w_pearson,q_w_spearman
0,DEG10000,57300.0,./DEG/DEG10000/DEG10000_data.csv,DEG,Thüringen,NaN,NaN,NaN,29646.0,29646.0,0.969240,0.976895
1,DEG10010,44700.0,./DEG/DEG10010/DEG10010_data.csv,DEG,Thüringen,NaN,NaN,NaN,22707.0,22707.0,NaN,NaN
2,DEG10020,57420.0,./DEG/DEG10020/DEG10020_data.csv,DEG,Thüringen,NaN,NaN,NaN,35490.0,35490.0,0.958767,0.962387
3,DEG10030,57650.0,./DEG/DEG10030/DEG10030_data.csv,DEG,Thüringen,NaN,NaN,NaN,12845.0,12845.0,0.502141,0.553354
4,DEG10040,57021.0,./DEG/DEG10040/DEG10040_data.csv,DEG,Thüringen,NaN,NaN,NaN,21246.0,21246.0,0.940139,0.977699
...,...,...,...,...,...,...,...,...,...,...,...,...
3655,DE215350,56113404,./DE2/DE215350/DE215350_data.csv,DE2,Bayern,8.2,4.456659e+06,3009715.476,16497.0,16497.0,0.558268,0.933178
3656,DE215360,56114000,./DE2/DE215360/DE215360_data.csv,DE2,Bayern,14.1,4.457008e+06,3011664.641,20880.0,20880.0,0.996458,0.985672
3657,DE215370,56122008,./DE2/DE215370/DE215370_data.csv,DE2,Bayern,84.3,4.465052e+06,3016792.655,23072.0,23072.0,0.399310,0.898170
3658,DE215380,56143008,./DE2/DE215380/DE215380_data.csv,DE2,Bayern,92.4,4.462157e+06,3021202.262,23440.0,23440.0,0.547284,0.871654


In [7]:
for NUTS in util._NUTS_LVL2_NAMES.keys():    
    # process this federal state
    with Bundesland(NUTS) as bl:    
        pearson = []
        spearman = []

        # get the metadata
        meta = bl.metadata

        # load the Data-report for each
        for camels_id in tqdm(meta.camels_id.values):
            p = os.path.join(bl.base_path, 'reports', f'{camels_id}.json')
            
            # check if the report exists
            if not os.path.exists(p):
                pearson.append(None)
                spearman.append(None)
                continue
            
            with open(p, 'r') as f:
                report = json.load(f)
            
            try:
                q = [o for o in report['correlations']['pearson'] if o['q'] == 1.0][0]
                w = [o for o in report['correlations']['pearson'] if o['w'] == 1.0][0]
                pearson.append(q['w'])
            except:
                pearson.append(None)
            
            try:
                q = [o for o in report['correlations']['spearman'] if o['q'] == 1.0][0]
                w = [o for o in report['correlations']['spearman'] if o['w'] == 1.0][0]
                spearman.append(q['w'])
            except:
                spearman.append(None)
        
        # all collected, return now
        corrs = pd.DataFrame({'camels_id': meta.camels_id.values, 'q_w_pearson': pearson, 'q_w_spearman': spearman})

        # update
        bl.update_metadata(corrs)

metadata = util.get_metadata()
metadata

100%|██████████| 540/540 [00:32<00:00, 16.61it/s]
0it [00:00, ?it/s]
100%|██████████| 382/382 [00:10<00:00, 34.82it/s] 
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 437/437 [00:22<00:00, 19.71it/s]
0it [00:00, ?it/s]
100%|██████████| 63/63 [00:04<00:00, 13.54it/s]


,camels_id,provider_id,camels_path,nuts_lvl2,federal_state,area,x,y,q_count,w_count,q_w_pearson,q_w_spearman
0,DEG10000,57300.0,./DEG/DEG10000/DEG10000_data.csv,DEG,Thüringen,NaN,NaN,NaN,29646.0,29646.0,0.969240,0.976895
1,DEG10010,44700.0,./DEG/DEG10010/DEG10010_data.csv,DEG,Thüringen,NaN,NaN,NaN,22707.0,22707.0,NaN,NaN
2,DEG10020,57420.0,./DEG/DEG10020/DEG10020_data.csv,DEG,Thüringen,NaN,NaN,NaN,35490.0,35490.0,0.958767,0.962387
3,DEG10030,57650.0,./DEG/DEG10030/DEG10030_data.csv,DEG,Thüringen,NaN,NaN,NaN,12845.0,12845.0,0.502141,0.553354
4,DEG10040,57021.0,./DEG/DEG10040/DEG10040_data.csv,DEG,Thüringen,NaN,NaN,NaN,21246.0,21246.0,0.940139,0.977699
...,...,...,...,...,...,...,...,...,...,...,...,...
3655,DE215350,56113404,./DE2/DE215350/DE215350_data.csv,DE2,Bayern,8.2,4.456659e+06,3009715.476,16497.0,16497.0,0.558268,0.933178
3656,DE215360,56114000,./DE2/DE215360/DE215360_data.csv,DE2,Bayern,14.1,4.457008e+06,3011664.641,20880.0,20880.0,0.996458,0.985672
3657,DE215370,56122008,./DE2/DE215370/DE215370_data.csv,DE2,Bayern,84.3,4.465052e+06,3016792.655,23072.0,23072.0,0.399310,0.898170
3658,DE215380,56143008,./DE2/DE215380/DE215380_data.csv,DE2,Bayern,92.4,4.462157e+06,3021202.262,23440.0,23440.0,0.547284,0.871654
